In [ ]:
from des_api import des_cipher
import sys
from test_feedback_mode_operation import *

# Feedback Mode Operations
For AES, DES, or any block cipher, encryption is performed on a block of b  bits. In the case of DES, $b =  64$ and in the case of AES, $b = 128$. However, it is possible to convert a block cipher into a stream cipher, using one of the three modes: 
- cipher feedback (CFB) mode, 
- output feedback (OFB) mode, and
- counter  (CTR) mode. 

A stream cipher eliminates the need to pad a message to be an integral number of blocks. It also can operate in real time. Thus, if a character stream is being transmitted, each character can be encrypted and transmitted immediately using a character-oriented stream cipher.

One desirable property of a stream cipher is that the ciphertext be of the same length as the plaintext. Thus, if 8-bit characters are being transmitted, each character should be encrypted to produce a ciphertext output of 8 bits. If more than 8 bits are produced, transmission capacity is wasted.

## Cipher Feedback Mode (CFB)

The following Figure depicts the CFB scheme. In the figure, it is assumed that the unit of transmission is $s$ bits; a common value is $s = 8$. As with CBC, the units of plaintext are chained together, so that the ciphertext of any plaintext unit is a function of all the preceding plaintext. In this case, rather than blocks of $b$ bits, the plaintext is divided into segments of $s$ bits.

<img src='fmo_images/cfb.png' width=50%>

**Encryption:** The input to the encryption function is a $b-bit$ shift register that is initially set to some initialization vector (IV). The leftmost (most significant) $s$  bits of the output of the encryption function are XORed with the first segment of plaintext $P_1$ to produce the first unit of ciphertext $C_1$, which is then transmitted. In addition, the contents of the shift register are shifted left by $s$ bits, and $C_1$  is placed in the rightmost (least significant) $s$ bits of the shift register. This process continues until all plaintext units have been encrypted.

**Decryption:** The same scheme is used, except that the received ciphertext unit is XORed with the output of the encryption function to produce the plaintext unit. Note that it is the encryption function that is used, not the decryption function.

In CFB encryption, like CBC encryption, the input block to each forward Cipher function (except the first) depends on the result of the previous forward Cipher function; therefore, multiple forward cipher operations cannot be performed in parallel. In CFB decryption, the required forward cipher operations can be performed in parallel if the input blocks are first constructed (in series) from the IV and the ciphertext.

**Exercise:** Implement `cfb_unit()` to perform each stage of the encryption and decryption in cfb model.  


In [ ]:
def cfb_unit(K, I, input_bits, s=8):
    
    # check that the size of the input_bits is s
    assert None
    
    # peform des encryption on I using K to produce Q
    Q = None
    
    # select s bits and discard b - s
    selected_bits = None
    
    # xor selected bits with input_text
    output_bits = None
    
    return output_bits
    
    
    
    
    

In [ ]:
print('Sanity Check (cfb_unit): ', test_cfb_unit(cfb_unit))

**Exercise:** implement `cfb()` to perform cipher feedback encryption and decryption. Note that

- `keys` parameter is a dictionary with `K` and `IV` keys that contain the values for key and initial vector.
- `input_blocks` parameter is a list where each element represents $P_i$ or $C_i$ depending on the encryption and decryption mode.

In [ ]:
def cfb(input_blocks, keys, mode, s=8):
    
    
    # make an empty list for output_blocks
    output_blocks = None
    
    # initally the I is IV
    I = None
    K = None
    
    for input_bits in input_blocks:
        
        # peform cfb_unit 
        output_bits = None
        
        # add output_bits to the output_blocks
        None
        
        # shift I to left s-bits and add cipher_bits to it
        None
            
    
    return output_blocks
    

In [ ]:
print('Sanity Check (cfb): ', test_cfb(cfb))

## Output Feedback (OFB) mode

The output feedback  (OFB) mode is similar in structure to that of CFB. For OFB, the output of the encryption function is fed back to become the input for encrypting the next block of plaintext. In CFB, the output of the XOR unit is fed back to become input for encrypting the next block. The other difference is that the OFB mode operates on full blocks of plaintext and ciphertext, whereas CFB operates on an s-bit subset.


<img src='fmo_images/ofb.png' width=50%>


As with CBC and CFB, the OFB mode requires an initialization vector. In the case of OFB, *the IV must be a nonce; that is, the IV must be unique to each execution of the encryption operation*. The reason for this is that the sequence of encryption output blocks, $O_i$ , depends only on the key and the IV and does not depend on the plaintext. Therefore, for a given key and IV, the stream of output bits used to XOR with the stream of plaintext bits is fixed. If two different messages had an identical block of plaintext in the identical position, then an attacker would be able to determine that portion of the $O_i$ stream.

One advantage of the OFB method is that bit errors in transmission do not propagate. For example, if a bit error occurs in $C_1$ , only the recovered value of $P_1$  is affected; subsequent plaintext units are not corrupted. With CFB, $C_1$  also serves as input to the shift register and therefore causes additional corruption downstream.


**Exercise:** Implement `ofb()` that implements  the OFB model. Note that 
- `keys` parameter is a dictionary with `K` and `NONCE` keys that contain the values for key and nonce values.
- `input_blocks` parameter is a list where each element represents $P_i$ or $C_i$ depending on the encryption and decryption mode.

In [ ]:
def ofb(input_blocks, keys):
    
    # make an empty list for output_blocks
    output_blocks = None
    
      
    # initial I is NONCE
    I = None
    
    for input_block in input_blocks:
        
        # perform des using K on input_bloc
        None
        
        # update I as O
        None
        
        # perform xor O with input_block
        None
        # append to outputs
        None
    
    return output_blocks
    

In [ ]:
print('Sanity Check (ofb): ', test_ofb(ofb))

## Counter  (CTR) mode
 
The following Figure depicts the CTR mode. A counter equal to the plaintext block size used. The only requirement stated in SP 800-38A is that the counter value must be different for each plaintext block that is encrypted. Typically, the counter is initialized to some value and then incremented by 1 for each subsequent block (modulo $2b$, where $b$  is the block size). 

<img src='fmo_images/ctr.png' width=50%>

For encryption, the counter is encrypted and then XORed with the plaintext block to produce the ciphertext block; there is no chaining. For decryption, the same sequence of counter values is used, with each encrypted counter XORed with a ciphertext block to recover the corresponding plaintext block. Thus, the initial counter value must be made available for decryption.

As with the OFB mode, the initial counter value must be a nonce; that is, $T_1$  must be different for all of the messages encrypted using the same key. Further, all $T_i$  values across all messages must be unique. If, contrary to this requirement, a counter value is used multiple times, then the confidentiality of all of the plaintext blocks corresponding to that counter value may be compromised. In particular, if any plaintext block that is encrypted using a given counter value is known, then the output of the encryption function can be determined easily from the associated ciphertext block. This output allows any other plaintext blocks that are encrypted using the same counter value to be easily recovered from their associated ciphertext blocks.

One way to ensure the uniqueness of counter values is to continue to increment the counter value by 1 across messages. That is, the first counter value of the each message is one more than the last counter value of the preceding message.


**Exercise:** Implement `ctr()` that implements the CTR model. Note that 
- `input_blocks` parameter is a list where each element represents $P_i$ or $C_i$ depending on the encryption and decryption mode.


In [ ]:
def ctr(input_blocks, key):
    
    # make an empty list for output_blocks
    output_blocks = None
    
    # initialize counter
    counter = 1
    
    for input_block in input_blocks:
        
        # convert to BitArray
        T = BitArray(format(counter,'#066b'))
        
        # encrupt T 
        None
        
        # xor O with input_block
        None
        
        # append output_block
        None
        
        # increament counter
        None
    
    return output_blocks
    

In [ ]:
print('Sanity Check (ctr): ', test_ctr(ctr))

### Grade
Run the following cell to calculate your grade.

In [ ]:
exercise_functions = get_module_functions(sys.modules[__name__])
evaluate(exercise_functions)